<a href="https://colab.research.google.com/github/joseph75Ghoat/ciitec2021/blob/master/CNN_B%C3%81SICA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cat vs. Dog CNN

## Carga y exploración de los datos

Vamos a descargar un conjunto de datos con 2000 imágenes de perros y datos.

**NOTE:** Este conjunto de datos es un subconjunto del dataset ["Dogs vs. Cats" dataset](https://www.kaggle.com/c/dogs-vs-cats/data) disponible en Kaggle, el dataset original contiene 25,000 imágenes.

In [ ]:
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
    -O /tmp/cats_and_dogs_filtered.zip

In [ ]:
import os
import zipfile

local_zip = '/tmp/cats_and_dogs_filtered.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp')
zip_ref.close()

Extraemos el contenido del .zip y posteriormente vamos a identificar los directorios relevantes.

In [ ]:
base_dir = '/tmp/cats_and_dogs_filtered'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

train_cats_dir = os.path.join(train_dir, 'cats')
train_dogs_dir = os.path.join(train_dir, 'dogs')

validation_cats_dir = os.path.join(validation_dir, 'cats')
validation_dogs_dir = os.path.join(validation_dir, 'dogs')

train_cat_fnames = os.listdir(train_cats_dir)
train_dog_fnames = os.listdir(train_dogs_dir)

Vamos a listar el número de imágenes en los conjuntos de datos para entrenamiento y validación.

In [ ]:
print('total training cat images:', len(os.listdir(train_cats_dir)))
print('total training dog images:', len(os.listdir(train_dogs_dir)))
print('total validation cat images:', len(os.listdir(validation_cats_dir)))
print('total validation dog images:', len(os.listdir(validation_dogs_dir)))

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

nrows = 4
ncols = 4

pic_index = 0

Vamos a utilizar Matplotlib para mostrar algunas imágenes del conjunto de datos

In [ ]:

fig = plt.gcf()
fig.set_size_inches(ncols * 4, nrows * 4)

pic_index += 8
next_cat_pix = [os.path.join(train_cats_dir, fname) 
                for fname in train_cat_fnames[pic_index-8:pic_index]]
next_dog_pix = [os.path.join(train_dogs_dir, fname) 
                for fname in train_dog_fnames[pic_index-8:pic_index]]

for i, img_path in enumerate(next_cat_pix+next_dog_pix):
  sp = plt.subplot(nrows, ncols, i + 1)
  sp.axis('Off') 

  img = mpimg.imread(img_path)
  plt.imshow(img)

plt.show()


## Construcción de la red neuronal


In [ ]:
from tensorflow.keras import layers
from tensorflow.keras import Model

In [ ]:
# Capa de entrada (Imágenes de 150 x 150 a color)
img_input = 

# Capa convolucional con 16 filtros de 3x3, relu non linearity
x = 
# Capa de max pooling 2 x 2
x = 

# Capa convolucional con 32 filtros de 3x3, relu non linearity
x = 
# Capa de max pooling 2 x 2
x = 

# Capa convolucional con 64 filtros de 3x3, relu non linearity
x = 
# Capa de max pooling 2 x 2
x = 

Después de pasar la imagen por el proceso Conv, NL, POOL x3 vamos a utilizar una red neuronal simple para el proceso de clasificación

In [ ]:
# Aplanar las capas
x = 

# Capa completamente conectada 512Neuronas
x = 

# Capa de salida, 1 sola neurona con activación binaria
output = 

# Creación del modelo
model = 

Resumen de la arquitectura

A continuación vamos a definir la estrategia de entrenamiento de nuestro modelo

In [ ]:
from tensorflow.keras.optimizers import RMSprop



### Preprocesamiento de las imágenes

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Reescalado de los valores en cada pixel
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

# Creación de los generadores
train_generator = train_datagen.flow_from_directory(
        train_dir, 
        target_size=(150, 150),  # Reescalado de las imágenes
        batch_size=20,
        class_mode='binary')

validation_generator = val_datagen.flow_from_directory(
        validation_dir,
        target_size=(150, 150),
        batch_size=20,
        class_mode='binary')

### Training
Let's train on all 2,000 images available, for 15 epochs, and validate on all 1,000 validation images. (This may take a few minutes to run.)

In [ ]:
history = 

### Visualización de las capas intermedias

In [ ]:
import numpy as np
import random
from tensorflow.keras.preprocessing.image import img_to_array, load_img


successive_outputs = [layer.output for layer in model.layers[1:]]
visualization_model = Model(img_input, successive_outputs)

cat_img_files = [os.path.join(train_cats_dir, f) for f in train_cat_fnames]
dog_img_files = [os.path.join(train_dogs_dir, f) for f in train_dog_fnames]
img_path = random.choice(cat_img_files + dog_img_files)

img = load_img(img_path, target_size=(150, 150)) 
x = img_to_array(img) 
x = x.reshape((1,) + x.shape)  


x /= 255

successive_feature_maps = visualization_model.predict(x)

layer_names = [layer.name for layer in model.layers[1:]]

for layer_name, feature_map in zip(layer_names, successive_feature_maps):
  if len(feature_map.shape) == 4:
    n_features = feature_map.shape[-1]
    size = feature_map.shape[1]
    display_grid = np.zeros((size, size * n_features))
    for i in range(n_features):
      x = feature_map[0, :, :, i]
      x -= x.mean()
      x /= x.std()
      x *= 64
      x += 128
      x = np.clip(x, 0, 255).astype('uint8')
      display_grid[:, i * size : (i + 1) * size] = x

    scale = 20. / n_features
    plt.figure(figsize=(scale * n_features, scale))
    plt.title(layer_name)
    plt.grid(False)
    plt.imshow(display_grid, aspect='auto', cmap='viridis')

### Evaluación del desempeño del modelo

In [ ]:
# Obtenemos los valores de exactitud del entrenamiento
acc = history.history['acc']
val_acc = history.history['val_acc']

# Obtenemos los valores de perdida
loss = history.history['loss']
val_loss = history.history['val_loss']

# Obtenemos el número de epocas
epochs = range(len(acc))

# Dibujamos el desempeño de nuestro modelo
plt.plot(epochs, acc)
plt.plot(epochs, val_acc)
plt.title('Training and validation accuracy')

plt.figure()

# Dibujamos el desempeño de nuestro modelo
plt.plot(epochs, loss)
plt.plot(epochs, val_loss)
plt.title('Training and validation loss')

### Predicción de una imágen cualquiera

In [ ]:
labels = list(validation_generator.class_indices.keys())

In [ ]:
labels

In [ ]:
import tensorflow as tf

img = tf.keras.utils.load_img(
    'mila.jpeg', target_size=(150, 150)
)
img_array = tf.keras.utils.img_to_array(img)
img_array = tf.expand_dims(img_array, 0)
prediction = model.predict(img_array)


print(labels[int(prediction > 0.5)])